In [34]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
#PATH = "C:\Users\mcveigh\Documents\PythonPC\chrome.exe"
#driver = webdriver.Chrome(PATH)
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re
import os
import sys

### List of URLs input to search

In [3]:
urls = ['https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006404',
       'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006406?emailalert=true',
        'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006401?emailalert=true'
       ]

### Selenium web scraper

In [4]:
#Function to scrape headlines using Selenium
def scrape_with_selenium(url):
    global title
    global footnote
    options = Options()
    options.headless = False  # Set to True for headless mode
    driver = webdriver.Chrome(options=options)

    #Navigate to the webpage
    driver.get(url)

    #Interact with the webpage using Selenium
    # Example: Click on a button that loads more articles
    #load_more_button = driver.find_element_by_css_selector('.load-more-button')
    #load_more_button.click()

    #Allow time for dynamic content to load (you may need to use WebDriverWait for more robust waiting)
    time.sleep(3)

    #Extract and print headlines after loading more content
    #headlines = driver.find_elements_by_css_selector('.card h3')
    #for headline in headlines:
       #print(headline.text)
    #print('test message')  
    #print(dir(driver))
    for element in driver.find_elements(By.CLASS_NAME, "item-meta-data__item-title"):
        #print(element.text)
        title = element.text
        #print(title)
    #for element in driver.find_elements(By.CLASS_NAME, "author-footnote-below-abstract"): #retrieves all footnotes
    for element in driver.find_elements(By.ID, "FN1"): #retrieves just footnote 1
        #print(element.text)
        footnote = element.text
        #print(footnote)
        #print(type(footnote))

    
    
    #Close the browser window
    driver.quit()
    return title, footnote



### Main body option 2 is working

In [26]:
pub_df = pd.DataFrame(columns=['PublishedName', 'Accessions'])
pd.set_option('display.max_columns', None)
for url in urls:
    scrape_with_selenium(url)
    print('title is', title)
    print('footnote is', footnote)

#find the organism names    
    match = [r'(\S+\s+){2}(?=sp. nov.)']
    regex = re.compile(r'\b(' + '|'.join(match) + r')\b')
    orgname = [m.group() for m in regex.finditer(title)]
    print(orgname)

    
#find the accessions
    pattern = [r'[A-Z]{2}\d{6}', r'[A-Z]{4}\d{8}', r'([A-Z]+)(_[A-Z]+)\d{6}', r'[A-Z]{6}\d{9}']
    regex = re.compile(r'\b(' + '|'.join(pattern) + r')\b')
    accessions = [m.group() for m in regex.finditer(footnote)]
    print(accessions)

#load data into pandas dataframe
    row_data = [orgname, accessions]
    length = len(pub_df)
    pub_df.loc[length] = row_data

title is Mycolicibacterium cyprinidarum sp. nov., a rapidly growing species isolated from diseased koi carp, Cyprinus carpio
footnote is Whole genome sequences of each strain are deposited in GenBank as BPRI00000000.1 (NGTWS0302), BPRJ00000000.1 (NGTWS1803T) and BPRH00000000.1 (NGTWSNA01). The 16S rRNA sequence of each isolate are described below: LC582326 (NGTWS0302), LC582327 (NGTWS1803T) and LC722566 (NGTWSNA01).
['Mycolicibacterium cyprinidarum ']
['BPRI00000000', 'BPRJ00000000', 'BPRH00000000', 'LC582326', 'LC582327', 'LC722566']
title is Kitasatospora cathayae sp. nov., a novel endophytic actinomycete isolated from the leaves of Cathaya argyrophylla
footnote is The NCBI accession numbers for the 16S rRNA gene sequence and genome sequence of strain HUAS 3-15T are OQ363168 and NZ_CP115450.1, respectively.
['Kitasatospora cathayae ']
['OQ363168', 'NZ_CP115450']
title is Gordonia prachuapensis sp. nov. and Gordonia sesuvii sp. nov., two novel actinobacteria isolated from mangrove sed

In [27]:
#missing a name from the pub with multiple names
pd.set_option('max_colwidth', None)
pub_df

,PublishedName,Accessions
0,[Mycolicibacterium cyprinidarum ],"[BPRI00000000, BPRJ00000000, BPRH00000000, LC582326, LC582327, LC722566]"
1,[Kitasatospora cathayae ],"[OQ363168, NZ_CP115450]"
2,"[Gordonia prachuapensis , Gordonia sesuvii ]","[OQ538201, PP145804, JAZDUE000000000, JAZDUF000000000]"


In [30]:
pub_df.explode(['PublishedName']).reset_index(drop=True)

,PublishedName,Accessions
0,Mycolicibacterium cyprinidarum,"[BPRI00000000, BPRJ00000000, BPRH00000000, LC582326, LC582327, LC722566]"
1,Kitasatospora cathayae,"[OQ363168, NZ_CP115450]"
2,Gordonia prachuapensis,"[OQ538201, PP145804, JAZDUE000000000, JAZDUF000000000]"
3,Gordonia sesuvii,"[OQ538201, PP145804, JAZDUE000000000, JAZDUF000000000]"


In [24]:
pub2_df = pub_df.explode(['Accessions']).reset_index(drop=True)
pub2_df

,PublishedName,Accessions
0,[Mycolicibacterium cyprinidarum ],BPRI00000000
1,[Mycolicibacterium cyprinidarum ],BPRJ00000000
2,[Mycolicibacterium cyprinidarum ],BPRH00000000
3,[Mycolicibacterium cyprinidarum ],LC582326
4,[Mycolicibacterium cyprinidarum ],LC582327
5,[Mycolicibacterium cyprinidarum ],LC722566
6,[Kitasatospora cathayae ],OQ363168
7,[Kitasatospora cathayae ],NZ_CP115450
8,"[Gordonia prachuapensis , Gordonia sesuvii ]",OQ538201
9,"[Gordonia prachuapensis , Gordonia sesuvii ]",PP145804


In [31]:
pub3_df = pub_df.explode(['PublishedName']).reset_index(drop=True)
pub3_df

,PublishedName,Accessions
0,Mycolicibacterium cyprinidarum,"[BPRI00000000, BPRJ00000000, BPRH00000000, LC582326, LC582327, LC722566]"
1,Kitasatospora cathayae,"[OQ363168, NZ_CP115450]"
2,Gordonia prachuapensis,"[OQ538201, PP145804, JAZDUE000000000, JAZDUF000000000]"
3,Gordonia sesuvii,"[OQ538201, PP145804, JAZDUE000000000, JAZDUF000000000]"


In [88]:
pub4_df = pub2_df.explode(['PublishedName']).reset_index(drop=True)
pub4_df.rename(columns={'Accessions' : 'accession'}, inplace=True)
pub4_df

,PublishedName,accession
0,Mycolicibacterium cyprinidarum,BPRI00000000
1,Mycolicibacterium cyprinidarum,BPRJ00000000
2,Mycolicibacterium cyprinidarum,BPRH00000000
3,Mycolicibacterium cyprinidarum,LC582326
4,Mycolicibacterium cyprinidarum,LC582327
5,Mycolicibacterium cyprinidarum,LC722566
6,Kitasatospora cathayae,OQ363168
7,Kitasatospora cathayae,NZ_CP115450
8,Gordonia prachuapensis,OQ538201
9,Gordonia sesuvii,OQ538201


In [55]:
df_unique = pub4_df.drop_duplicates(["Accessions"], keep="first")
df_unique

,PublishedName,Accessions
0,Mycolicibacterium cyprinidarum,BPRI00000000
1,Mycolicibacterium cyprinidarum,BPRJ00000000
2,Mycolicibacterium cyprinidarum,BPRH00000000
3,Mycolicibacterium cyprinidarum,LC582326
4,Mycolicibacterium cyprinidarum,LC582327
5,Mycolicibacterium cyprinidarum,LC722566
6,Kitasatospora cathayae,OQ363168
7,Kitasatospora cathayae,NZ_CP115450
8,Gordonia prachuapensis,OQ538201
10,Gordonia prachuapensis,PP145804


In [67]:
with open('acclist', 'w') as f:
    for text in df_unique['Accessions'].tolist():
        f.write(text + '\n')


In [68]:
os.system("/netopt/ncbi_tools64/bin/srcchk -i acclist -f taxname,taxid,strain,type -o acclist.taxdata")


0

In [86]:
taxdata_file_name = (r'acclist.taxdata')    
srcchk_df = pd.read_csv(taxdata_file_name, sep='\t', index_col=None, low_memory=False)
srcchk_df .rename(columns={'organism' : 'NCBIname','Unnamed: 4':'type'}, inplace=True)
srcchk_df['accession'] = srcchk_df['accession'].astype(str).apply(lambda x: x.replace('.1',''))
srcchk_df 

,accession,NCBIname,taxid,strain,type
0,BPRI00000000,Mycolicibacterium cyprinidarum,2860311,NGTWS0302,NaN
1,BPRJ00000000,Mycolicibacterium cyprinidarum,2860311,NGTWS1803,NaN
2,BPRH00000000,Mycolicibacterium sp. NGTWSNA01,2834227,NGTWSNA01,NaN
3,LC582326,Mycolicibacterium cyprinidarum,2860311,NGTWS0302,NaN
4,LC582327,Mycolicibacterium cyprinidarum,2860311,NGTWS1803,NaN
5,LC722566,Mycolicibacterium cyprinidarum,2860311,NGTWSNA01,NaN
6,OQ363168,Kitasatospora cathayae,3004092,HUAS 3-15,NaN
7,NZ_CP115450,Kitasatospora cathayae,3004092,HUAS 3-15,NaN
8,OQ538201,Gordonia prachuapensis,3115651,PKS22-38,NaN
9,PP145804,Gordonia sesuvii,3116777,LSe1-13,NaN


In [90]:
combine_df=pd.merge(left=pub4_df, right=srcchk_df, left_on='accession', right_on='accession', how = 'outer')
combine_df

,PublishedName,accession,NCBIname,taxid,strain,type
0,Mycolicibacterium cyprinidarum,BPRI00000000,Mycolicibacterium cyprinidarum,2860311,NGTWS0302,NaN
1,Mycolicibacterium cyprinidarum,BPRJ00000000,Mycolicibacterium cyprinidarum,2860311,NGTWS1803,NaN
2,Mycolicibacterium cyprinidarum,BPRH00000000,Mycolicibacterium sp. NGTWSNA01,2834227,NGTWSNA01,NaN
3,Mycolicibacterium cyprinidarum,LC582326,Mycolicibacterium cyprinidarum,2860311,NGTWS0302,NaN
4,Mycolicibacterium cyprinidarum,LC582327,Mycolicibacterium cyprinidarum,2860311,NGTWS1803,NaN
5,Mycolicibacterium cyprinidarum,LC722566,Mycolicibacterium cyprinidarum,2860311,NGTWSNA01,NaN
6,Kitasatospora cathayae,OQ363168,Kitasatospora cathayae,3004092,HUAS 3-15,NaN
7,Kitasatospora cathayae,NZ_CP115450,Kitasatospora cathayae,3004092,HUAS 3-15,NaN
8,Gordonia prachuapensis,OQ538201,Gordonia prachuapensis,3115651,PKS22-38,NaN
9,Gordonia sesuvii,OQ538201,Gordonia prachuapensis,3115651,PKS22-38,NaN


### taxonerd test

In [2]:
from taxonerd import TaxoNERD
taxonerd = TaxoNERD(prefer_gpu=False)
nlp = taxonerd.load(model="en_ner_eco_md", exclude=[], linker="taxref", threshold=0.7)
nlp.pipe_names

Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


['tok2vec',
 'tagger',
 'attribute_ruler',
 'lemmatizer',
 'lower_case_lemmas',
 'pysbd_sentencizer',
 'parser',
 'ner',
 'taxo_abbrev_detector',
 'taxon_linker']

### option to use biopython to retrieve orgnames based on accessions

In [39]:
import Bio
from Bio import Entrez
from Bio import SeqIO

In [40]:
handle = Entrez.efetch(db="nucleotide", id="AY851612", rettype="gb", retmode="text")
x = SeqIO.read(handle, 'genbank')
x.annotations['organism']

'Austrocylindropuntia subulata'